In [277]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
appended_data = []
for year in range(1985, 2020):
    result = pd.DataFrame()
    url = 'https://www.basketball-reference.com/awards/awards_{}.html'.format(year)
    url2 = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'.format(year)
    url3 = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'.format(year)
    
    
    #**********************EXTRACT DATA FROM FIRST URL**********************
    temp = pd.read_html(url, header=[1])
    temp[0].dropna(axis=1,inplace=True)
    temp = temp[0][0:10]
    temp = temp.drop('Rank', 1)
    temp.insert(1, 'Year', year)
    temp.insert(2, 'MVP', [1,0,0,0,0,0,0,0,0,0])
    temp.columns = temp.columns.str.strip()
    temp = temp.round(4)
    #**********************EXTRACT DATA FROM SECOND URL**********************
    temp2 = pd.read_html(url2)
    temp2 = temp2[0].dropna(axis=1,how='all')
    temp2['Player'] = temp2['Player'].str.replace('*','')
    temp2.columns = temp2.columns.str.strip()
    temp2 = temp2[temp2.Pos != 'Pos'] #get rid of random header rows
    temp2 = temp2.round(4)
    #Find intersecting columns
    cols = list(np.intersect1d(temp.columns, temp2.columns))
    #Remove cols temporarily that don't involve numbers because casting
    #will add them back later when doing 'on = cols'
    cols.remove('Player')
    cols.remove('Tm')
    #need to convert cols to float
    temp[cols] = temp[cols].astype(float)
    temp2[cols] = temp2[cols].astype(float) 
    
    #add qualitative analysis back for on = within merge function
    cols.append('Player')
    cols.append('Tm')
    
    #MERGE TEMP AND TEMP2
    result = temp.merge(temp2, how='inner', on=cols)
    if (year == 1985):
        result.to_csv('csv/TEMP1_2.csv')
    #**********************EXTRACT DATA FROM THIRD URL**********************
    temp3 = pd.read_html(url3)
    temp3 = temp3[0].dropna(axis=1, how='all')
    temp3 = temp3[temp3.Pos != 'Pos']
    temp3['Player'] = temp3['Player'].str.replace('*','')
    temp3 = temp3.drop('Rk', 1)
    temp3.columns = temp3.columns.str.strip()
    temp3 = temp3.rename(columns={"MP": "TOT_MP"})
    temp3 = temp3.round(4)
    #Find intersecting columns
    cols = list(np.intersect1d(result.columns, temp3.columns))

    #Remove cols temporarily that don't involve numbers because casting
    #will add them back later when doing 'on = cols'
    cols.remove('Player')
    cols.remove('Tm')
    cols.remove('Pos')
    #need to convert cols to float
    temp3[cols] = temp3[cols].astype(float) 
    temp3.to_csv('csv/temp3.csv')
    #add qualitative analysis back for on = within merge function
    cols.append('Player')
    cols.append('Tm')
    cols.append('Pos')
    result = result.merge(temp3, how='inner', on=cols)
    #add to list of data
    appended_data.append(result)
df = pd.concat(appended_data)
df.to_csv('csv/historical_mvp_data.csv')

In [278]:
df.count()

Player     350
Year       350
MVP        350
Age        350
Tm         350
First      350
Pts Won    350
Pts Max    350
Share      350
G          350
MP         350
PTS        350
TRB        350
AST        350
STL        350
BLK        350
FG%        350
FT%        350
WS         350
WS/48      350
Rk         350
Pos        350
GS         350
FG         350
FGA        350
3P         350
3PA        350
3P%        344
2P         350
2PA        350
2P%        350
eFG%       350
FT         350
FTA        350
ORB        350
DRB        350
TOV        350
PF         350
TOT_MP     350
PER        350
TS%        350
3PAr       350
FTr        350
ORB%       350
DRB%       350
TRB%       350
AST%       350
STL%       350
BLK%       350
TOV%       350
USG%       350
OWS        350
DWS        350
OBPM       350
DBPM       350
BPM        350
VORP       350
dtype: int64

In [276]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
appended_data = []
for year in range(1987, 1988):
    result = pd.DataFrame()
    url = 'https://www.basketball-reference.com/awards/awards_{}.html'.format(year)
    url2 = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'.format(year)
    url3 = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'.format(year)
    
    
#     #**********************EXTRACT DATA FROM FIRST URL**********************
#     temp = pd.read_html(url, header=[1])
#     temp[0].dropna(axis=1,inplace=True)
#     temp = temp[0][0:10]
#     temp = temp.drop('Rank', axis=1)
#     temp = temp.drop('FG%', axis=1)
#     temp.insert(1, 'Year', year)
#     temp.insert(2, 'MVP', [1,0,0,0,0,0,0,0,0,0])
#     temp.columns = temp.columns.str.strip()
    
    #**********************EXTRACT DATA FROM SECOND URL**********************
#     temp2 = pd.read_html(url2)
#     temp2 = temp2[0].dropna(axis=1)
#     temp2['Player'] = temp2['Player'].str.replace('*','')
#     temp2.columns = temp2.columns.str.strip()
#     temp2 = temp2[temp2.Pos != 'Pos']
#    # temp2 = temp2.drop('FG%', axis=1)
    
#     #Find intersecting columns
#     cols = list(np.intersect1d(temp.columns, temp2.columns))
#     #Remove cols temporarily that don't involve numbers because casting
#     #will add them back later when doing 'on = cols'
#     cols.remove('Player')
#     cols.remove('Tm')

#     #need to convert cols to float
#     temp[cols] = temp[cols].astype(float)
#     temp2[cols] = temp2[cols].astype(float) 
    
#     #add qualitative analysis back for on = within merge function

#     #MERGE TEMP AND TEMP2
#     result = temp.merge(temp2, how='inner', on=cols)
#     print(temp2.columns)
#     if (year == 1985):
#         result.to_csv('csv/TEMP1_2.csv')
    temp3 = pd.read_html(url3)
    temp3 = temp3[0].dropna(axis=1, )
    temp3 = temp3[temp3.Pos != 'Pos']
    temp3['Player'] = temp3['Player'].str.replace('*','')
    temp3 = temp3.drop('Rk', 1)
    temp3.columns = temp3.columns.str.strip()
    temp3 = temp3.rename(columns={"MP": "TOT_MP"})
    temp3 = temp3.round(4)
    print(temp3.columns)
    #Find intersecting columns
    #cols = list(np.intersect1d(result.columns, temp3.columns))
    
    #Remove cols temporarily that don't involve numbers because casting
    #will add them back later when doing 'on = cols'
#     cols.remove('Player')
#     cols.remove('Tm')
#     cols.remove('Pos')
    #need to convert cols to float
    temp3[cols] = temp3[cols].astype(float) 
    display(temp3)
    temp3.to_csv('csv/temp3.csv')
    #add qualitative analysis back for on = within merge function
    cols.append('Player')
    cols.append('Tm')
    cols.append('Pos')
    display(result)
    result = result.merge(temp3, how='inner', on=cols)

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'TOT_MP', 'PER', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP'],
      dtype='object')


KeyError: "['PTS', 'BLK', 'TRB', 'AST', 'MP', 'STL'] not in index"